# First Step

In first step, model learns based on natural conditions and without applying self effects.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical

(x_train_origin, y_train_origin), (x_test_origin, y_test_origin) = keras.datasets.mnist.load_data()

# Resize data and scale them 
x_train_origin = x_train_origin.reshape(-1, 28*28)
x_train_origin = x_train_origin / 255.0
x_test_origin = x_test_origin.reshape(-1, 28*28)
x_test_origin = x_test_origin / 255.0

# define neural network
nature_model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(784,)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

# model settings
nature_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Array to store the weights of neurons and edges during training
neuron_weights = []
connection_weights = []

# Callback to record the weights of neurons and edges during training 
class SaveWeightsCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        neuron_weights.append(nature_model.layers[0].get_weights()[0].copy())
        connection_weights.append(nature_model.layers[1].get_weights()[0].copy())


number_of_epochs = 10
# train model using callback
history = nature_model.fit(x_train_origin, y_train_origin, epochs=number_of_epochs, validation_data=(x_test_origin, y_test_origin), callbacks=[SaveWeightsCallback()])

# print accuracy and loss of trained model
loss, accuracy = nature_model.evaluate(x_test_origin, y_test_origin)
print(f'Loss: {loss}, Accuracy: {accuracy}')

# show accuracy change in learning duration
plt.figure(figsize = (8, 6))
plt.errorbar(range(1, number_of_epochs+1), history.history['accuracy'],  yerr=0.0025, ecolor='red', color = 'b', capsize=2, marker='o',
              markersize=2, linestyle='-', linewidth=1.5, elinewidth=1)
plt.errorbar(range(1, number_of_epochs+1), history.history['val_accuracy'], yerr=0.0025, ecolor='red', color = 'orange', capsize=2, marker='o',
              markersize=2, linestyle='-', linewidth=1.5, elinewidth=1)

plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.yticks(np.arange(0.91, 1, step=0.05))
plt.xticks(np.arange(1, number_of_epochs+1, step=1))
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


# Second step

In second step, model continue learning based on both natural conditions and self effects but self coeff is greater that nature, and it represents habit...

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
import random

# coefficient of psychon(self) and nature that represent their power
psychon_coeff = 1.005
nature_coeff = 1.002

# filter train and test data just for number 1
x_train_origin = x_train_origin[y_train_origin == 1]
y_train_origin = y_train_origin[y_train_origin == 1]

x_test_origin = x_test_origin[y_test_origin == 1]
y_test_origin = y_test_origin[y_test_origin == 1]



# function to update probability for label prediction in each step for learning based on psychon(self) and nature coeff
def label_probability(psychon_coeff, nature_coeff, nature_label, psychon_label, nature_label_prob, psychon_label_prob):
    diff = psychon_coeff-nature_coeff
    adding_result = diff * nature_label_prob

    if nature_label_prob-adding_result>0:
        psychon_label_prob += adding_result
        nature_label_prob -= adding_result
    else:
        print("the probability of old label is smaller than diff.")

    label_list = range(10)
    weights = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    to_divide = (1-psychon_label_prob-nature_label_prob)
    if to_divide>0:
        to_divide = to_divide/8
    else:
        to_divide = 0
    weight_sum = 0
    for i in range(1, 10):
        if i == nature_label:
            weights[i] = nature_label_prob
            weight_sum += weights[i]
        elif i == psychon_label:
            weights[i] = psychon_label_prob
            weight_sum += weights[i]
        else:
            weights[i] = to_divide
            weight_sum += weights[i]
    weights[0] = 1-weight_sum
    # print("weights: ", weights)
    # print(sum(weights))
    label = np.random.choice(label_list, 1, p = weights)
    # label = np.argmax(weights)
    # print("label: ", label[0])
    # print("nature_label_prob: ", nature_label_prob)
    # print("psychon_label_prob: ", psychon_label_prob)
    return label[0], nature_label_prob, psychon_label_prob

model = nature_model
nature_prob_list = []
psychon_prob_list = []
old_label = 1
new_label = 1
psychon_label = 9
nature_label = 1
prediction = model.predict(x_test_origin, verbose=0)[0]
nature_label_prob = nature_prob = prediction[nature_label]
psychon_label_prob = psychon_prob = prediction[psychon_label]
nature_prob_list.append(nature_prob)
psychon_prob_list.append(psychon_prob)

number_of_steps = 2000
# Train 1 by 1
for i in range(number_of_steps):
    old_label = new_label
    new_label, nature_label_prob, psychon_label_prob = label_probability(psychon_coeff, nature_coeff, nature_label, psychon_label, nature_label_prob, psychon_label_prob)
    x_train = np.array([x_train_origin[i]])
    y_train = np.array([new_label])
    # print(y_train)
    x_test = x_test_origin
    y_test = y_test_origin[y_test_origin==1]
    y_test[y_test==1] = new_label
    # print("y_test_origin", y_test_origin)
    # print("y_test", y_test)
    prediction = model.predict(x_test_origin, verbose=0)[0]
    nature_prob = prediction[nature_label]
    psychon_prob = prediction[psychon_label]
    nature_prob_list.append(nature_prob)
    psychon_prob_list.append(psychon_prob)
    # print("nature_prob", nature_prob)
    # print("psychon_prob", psychon_prob)
    # print((y_test == y_test_origin[y_test_origin==1]).all())
    # train model using callback
    model.fit(x_train, y_train, epochs=1, validation_data=(x_test, y_test), verbose=0) #, callbacks=[cp_callback]



print("nature_prob_list: ", nature_prob_list)
print("psychon_prob_list: ", psychon_prob_list)
# show accuracy change in learning duration
plt.figure(figsize = (12, 6))
plt.plot(nature_prob_list)
plt.plot(psychon_prob_list)
saturation_fit = 0.99 * np.ones(number_of_steps-800)
plt.plot(range(801, number_of_steps+1), saturation_fit, linestyle='-.', color='g')
plt.title('Change Probability')
plt.ylabel('Probability', labelpad=10)
plt.xlabel('Step', labelpad=10)
plt.yticks(np.arange(0, 1.05, step=0.05))
plt.xticks(np.arange(number_of_steps, step=200))
plt.legend(['Nature_label'+ " ($N_{coeff}$"+ " = {})".format(nature_coeff), 'Psychon_label'+ " ($P_{coeff}$"+ " = {})".format(psychon_coeff), 'Saturation'], loc='upper left')
plt.savefig('{}_and_{}.png'.format(psychon_coeff, nature_coeff), dpi=600, transparent=False)
plt.show()


# Third step

In third step, it model again will leran based on natural conditions and it represent forgetting the hapit...

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
import random

# coefficient of psychon(self) and nature that represent their power
psychon_coeff = 1
nature_coeff = 1.002

# function to update probability for label prediction in each step for learning based on psychon(self) and nature coeff
def label_probability(psychon_coeff, nature_coeff, nature_label, psychon_label, nature_label_prob, psychon_label_prob):
    diff = psychon_coeff-nature_coeff
    adding_result = diff * nature_label_prob

    if psychon_label_prob+adding_result>0:
        psychon_label_prob += adding_result
        nature_label_prob -= adding_result
    else:
        print("the probability of old label is smaller than diff.")

    label_list = range(10)
    weights = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    sum_two_label = psychon_label_prob+nature_label_prob
    to_divide = 1-sum_two_label
    if to_divide>0:
        to_divide = to_divide/8
    elif to_divide<0:
        psychon_label_prob /= sum_two_label
        nature_label_prob /= sum_two_label
    else:
        to_divide = 0
    weight_sum = 0
    for i in range(1, 10):
        if i == nature_label:
            weights[i] = nature_label_prob
            weight_sum += weights[i]
        elif i == psychon_label:
            weights[i] = psychon_label_prob
            weight_sum += weights[i]
        else:
            weights[i] = to_divide
            weight_sum += weights[i]
    if weight_sum<1:
      weights[0] = 1-weight_sum

    # print("weights: ", weights)
    # print(sum(weights))
    label = np.random.choice(label_list, 1, p = weights)
    # label = np.argmax(weights)
    # print("label: ", label[0])
    # print("nature_label_prob: ", nature_label_prob)
    # print("psychon_label_prob: ", psychon_label_prob)
    return label[0], nature_label_prob, psychon_label_prob

nature_prob_list = []
psychon_prob_list = []
old_label = 1
new_label = 1
psychon_label = 9
nature_label = 1
prediction = model.predict(x_test_origin, verbose=0)[0]
nature_label_prob = nature_prob = prediction[nature_label]
psychon_label_prob = psychon_prob = prediction[psychon_label]
nature_prob_list.append(nature_prob)
psychon_prob_list.append(psychon_prob)


number_of_steps = 4500
# Train 1 by 1
for i in range(number_of_steps):
  try:
    old_label = new_label
    new_label, nature_label_prob, psychon_label_prob = label_probability(psychon_coeff, nature_coeff, nature_label, psychon_label, nature_label_prob, psychon_label_prob)
    x_train = np.array([x_train_origin[i]])
    y_train = np.array([new_label])
    # print(y_train)
    x_test = x_test_origin
    y_test = y_test_origin[y_test_origin==1]
    y_test[y_test==1] = new_label
    # print("y_test_origin", y_test_origin)
    # print("y_test", y_test)
    prediction = model.predict(x_test_origin, verbose=0)[0]
    nature_prob = prediction[nature_label]
    psychon_prob = prediction[psychon_label]
    nature_prob_list.append(nature_prob)
    psychon_prob_list.append(psychon_prob)
    # print("nature_prob", nature_prob)
    # print("psychon_prob", psychon_prob)
    # print((y_test == y_test_origin[y_test_origin==1]).all())
    # train model using callback
    model.fit(x_train, y_train, epochs=1, validation_data=(x_test, y_test), verbose=0) #, callbacks=[cp_callback]
  except Exception as ex:
    print(ex)



print("nature_prob_list: ", nature_prob_list)
print("psychon_prob_list: ", psychon_prob_list)
# show accuracy change in learning duration
plt.figure(figsize = (12, 6))
plt.plot(nature_prob_list)
plt.plot(psychon_prob_list)
saturation_fit = 0.01 * np.ones(number_of_steps-1500)
plt.plot(range(1501, number_of_steps+1), saturation_fit, linestyle='-.', color='g')
plt.title('Change Probability')
plt.ylabel('Probability', labelpad=10)
plt.xlabel('Step', labelpad=10)
plt.yticks(np.arange(0, 1.05, step=0.05))
plt.xticks(np.arange(number_of_steps, step=300))
plt.legend(['Nature_label'+ " ($N_{coeff}$"+ " = {})".format(nature_coeff), 'Psychon_label'+ " ($P_{coeff}$"+ " = {})".format(psychon_coeff), 'Saturation'], loc='upper left')
plt.savefig('{}_and_{}.png'.format(psychon_coeff, nature_coeff), dpi=600, transparent=False)
plt.show()

and we can go further and change nature and psychon(self) coefficient to see what happens and what will model learns...